In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score, mean_squared_log_error, mean_squared_error,mean_absolute_error
import datetime as dt
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import preparing_data as F
import ml_module as ml

In [2]:
#data=pd.read_pickle("./dataframe/df_20210912_133401_ONLY_RISKY_EVENTS-6-withFE.pkl")
#data=pd.read_pickle("./dataframe/df_20210914_003013_equally_distributed.pkl")
#data=pd.read_pickle("dataframe/_PRUEBA_df_20211225_143242.pkl") #dataframe usado en 2022
#data=pd.read_pickle("dataframe/_PRUEBA_df_filtered_20220114_202454.pkl") #dataframe filtrado usado en 2022
data=pd.read_pickle("dataframe/_PRUEBA_df_filtered_20220115_112153.pkl")
data.reset_index(inplace=True)
data.drop(['index'], inplace=True, axis=1)
print(data.shape)
data.head()

(5394, 85)


,__time_to_tca,MISS_DISTANCE,RELATIVE_SPEED,RELATIVE_POSITION_R,RELATIVE_POSITION_T,RELATIVE_POSITION_N,RELATIVE_VELOCITY_R,RELATIVE_VELOCITY_T,RELATIVE_VELOCITY_N,COLLISSION_PROBABILITY,...,OBJECT2_CORR_CNDOT_TDOT,PC_trend_1,PC_trend_3,PC_gradient_1,PC_gradient_3,MD_trend_1,MD_trend_3,MD_gradient_1,MD_gradient_3,TARGET_PC
0,5.775947,568.0,2001.0,-20.9,-562.8,-75.9,0.8,-268.6,1983.8,-5.415895,...,-0.054742,0.226170,0.773872,0.652610,0.763062,-123.0,144.0,-354.914301,141.988456,-5.345246
1,5.420762,611.0,2001.0,-19.9,-605.4,-81.8,0.8,-268.6,1983.8,-5.345246,...,0.029148,0.070649,0.237282,0.198907,0.234313,43.0,50.0,121.063636,49.374243,-4.792366
2,5.119489,576.0,2001.0,-9.6,-571.2,-77.4,0.8,-268.6,1983.8,-4.792366,...,-0.030920,0.552880,0.849700,1.835148,0.847141,-35.0,-115.0,-116.173757,-114.653689,-4.208450
3,4.750068,328.0,2001.0,2.7,-325.3,-43.7,0.5,-268.6,1983.8,-4.208450,...,-0.104064,0.583916,1.207445,1.580622,1.176985,-248.0,-240.0,-671.319754,-233.945596,-4.049879
4,4.087221,56.0,2001.0,9.2,-55.1,-7.3,0.2,-268.6,1983.8,-4.049879,...,-0.117181,0.158571,1.295367,0.239227,0.971374,-272.0,-555.0,-410.351242,-416.185112,-5.289798


In [3]:
data.shape

(5394, 85)

In [4]:
train, test = train_test_split(data, test_size=0.30, random_state=42)

In [5]:
print("Train dataframe dimension {} x {}".format(train.shape[0],train.shape[1]))
print("Test dataframe dimension {} x {}".format(test.shape[0],test.shape[1]))

Train dataframe dimension 3775 x 85
Test dataframe dimension 1619 x 85


In [6]:
columnt_to_predict_name="TARGET_PC"

In [7]:
Y_train = train[columnt_to_predict_name]
X_train= train.drop([columnt_to_predict_name], axis=1)
Y_test = test[columnt_to_predict_name]
X_test= test.drop([columnt_to_predict_name], axis=1)


In [8]:
X = X_train
y = Y_train

In [ ]:
model=ml.create_and_validate_model(X, y, X_test, Y_test,
                        init_iter=5, n_iters=300, random_state=77, seed = 101,num_iterations=100,
                        evalm="lgb_r2",
                        hp_metric="regression_L2")

In [9]:
#iteration for different numbers of tress
for i in [300,500,800,1000,1500,2000,3000,4000,5000]:
    ml.create_and_validate_model(X, y, X_test, Y_test,
                            init_iter=5, n_iters=300, random_state=77, seed = 101,num_iterations=i,
                            evalm="lgb_r2",
                            hp_metric="regression_L2")

|   iter    |  target   | baggin... | featur... | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.782    |  1.0      |  0.6138   |  0.03793  |  8.115    |  12.18    |  0.07901  |  40.49    |
|  2        |  0.7727   |  1.0      |  0.2922   |  0.02773  |  10.2     |  27.88    |  0.08383  |  71.44    |
|  3        |  0.777    |  1.0      |  0.3248   |  0.03557  |  10.38    |  11.43    |  0.07496  |  55.37    |
|  4        |  0.7484   |  1.0      |  0.1395   |  0.01533  |  7.534    |  28.78    |  0.007313 |  52.97    |
|  5        |  0.7639   |  1.0      |  0.2216   |  0.02779  |  10.55    |  10.9     |  0.08247  |  34.25    |
|  6        |  0.7792   |  1.0      |  0.3609   |  0.02751  |  8.922    |  23.64    |  0.04065  |  86.39    |
|  7        |  0.7818   |  1.0      |  0.874    |  0.03829  |  11.08    |  30.32    |  0.09475  |  87.88    |
|  8      

In [ ]:
# ml.create_and_validate_model(X, y,
#                         init_iter=5, n_iters=1500, random_state=77, seed = 101,num_iterations=300,evalm="lgb_adjusted_r2")

In [ ]:
# ml.create_and_validate_model(X, y,
#                         init_iter=5, n_iters=1500, random_state=77, seed = 101,num_iterations=300,evalm="lgb_rmse",hp_metric="rmse")

In [ ]:
aux=ml.load_and_validate_model("param_lgb_r2_only_risky_events",X,y,X_test,Y_test)
df_r2=aux[1]
aux=ml.load_and_validate_model("param_lgb_adjusted_r2_only_risky_events",X,y,X_test,Y_test)
df_r2_adj=aux[1]

In [ ]:
#df=model[1]

In [ ]:
df=df_r2

In [ ]:
df.head()

In [ ]:
df[df["y_true_10"]>0.00001]

In [ ]:
df[df["y_true_10"]>0.0001]

In [ ]:
df_r2_adj[df_r2_adj["y_true_10"]>0.0001]

In [ ]:
pr2=df[df["y_true_10"]>0.0001]
pr2_adj=df_r2_adj[df_r2_adj["y_true_10"]>0.0001]

In [ ]:
pd.concat([pr2,pr2_adj],axis=1)

In [ ]:
data = [pr2["y_true"], pr2["y_predicted"],pr2_adj["y_predicted"]]
headers = ["PC_true", "PC_predic_r2","PC_predic_r2_ajus"]
df3 = pd.concat(data, axis=1, keys=headers)
df3

In [ ]:
mylist=["ZONA DE MANIOBRA","ZONA DE MANIOBRA","ZONA DE MANIOBRA","ZONA DE MANIOBRA","ZONA DE MANIOBRA","ZONA DE MANIOBRA","ZONA DE ALERTA","ZONA DE MANIOBRA"]
df3["label"]=np.array(mylist)

In [ ]:
df3=np.round(df3,4)

In [ ]:
#df3.to_csv("./report/pc_predichas.csv",index=False)